### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 3 мая 2021, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 03 мая, -4 балла после 08:30 10 мая, -6 баллов после 08:30 17 мая, -8 баллов после 08:30 24 мая.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0221, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier


In [66]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1
 
    def __init__(self, min_samples_split=2, max_depth=5, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
            
        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
 
    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - ((np.sum(np.hstack((l_c*l_c/l_s, r_c*r_c/r_s)),
                            axis=1))/(l_s[0]+r_s[0]))
   

    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(np.hstack((l_c * np.log2(l_c / l_s),
                                  r_c * np.log2(r_c / r_s)),
                                 axis=1))/(l_s[0] + r_s[0])

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (np.max(l_c, axis=1) + np.max(r_c, axis=1)) / (l_s + r_s)
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]
 
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = list(range(n_feature))
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]
 
    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
 
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]
    
    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        """
        Находим оптимальный признак и порог для сплита
        Здесь используемые разные impurity в зависимости от self.criterion
        """
        # сортировка x и y
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        cut_size = round(self.min_samples_split / 2) - 1
        # делим наши y 
        if (cut_size != 0):
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        else:
            splitted_sorted_y = sorted_y
        # здесь находим индексы элементов, у которых сосед совпадает с ними, т.е. находим правых соседей пар одинаковых эл-тов
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (cut_size + 1)
        if len(r_border_ids) == 0:
            return float('+inf'), None
 
        # Получаем количество одинаковых идущих подряд элементов в интервале, потом создаем матрицу, где 1 там, где произошла смена
        eq_el_count = r_border_ids - np.append([cut_size], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        # Затем делаем матрицу количества классов для каждого интервала
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:cut_size], minlength=class_number)
 
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y, minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes
 
        # Считаем Gain, берем наименьший
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
 
        # left_el_id - индекс наилучшего разбиения
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0
 
 
    def fit(self, x, y):
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0)
        return self


    def __fit_node(self, x, y, node_id, depth, ind=-1):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        if (ind == -1):
            self.feature_importances_ = np.zeros(x.shape[1])
            self.Size_ = x.shape[1]    
        p = np.bincount(y, minlength=self.num_class) / x.shape[0]
        moda = np.argmax(p)
        # Критерии останова
        if self.max_depth is not None and depth >= self.max_depth or\
                    y.shape[0] < self.min_samples_split or\
                    self.sufficient_share <= p[moda]:
            self.tree[node_id] = (self.LEAF_TYPE, moda, p)
            return 

        ids = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, idx], y) for idx in ids])
        best = thresholds[:, 0].argmin()
        thr = thresholds[best, 1]
        imp_curr = thresholds[best, 0]
        
        if (thr is None):
            self.tree[node_id] = (self.LEAF_TYPE, moda, p)
            return

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, ids[best], thr)
        if (x_l.shape[0] == 0 or x_r.shape[0] == 0):
            self.tree[node_id] = (self.LEAF_TYPE, moda, p)
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, ids[best], thr)

        if self.G_function == self.__gini:
            imp_prev = 1 - np.sum(p ** 2)
        elif self.G_function == self.__entropy:
            imp_prev = -np.sum(p * np.log2(p))
        else:
            imp_prev = 1 - np.max(p)
        self.feature_importances_[ids[best]] += x.shape[0] * (imp_prev - imp_curr) / self.Size_
        
        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, 0)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, 0)
        
    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
 
    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
 
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in X])
 
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])
 
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)
   
    #my_clf.fit(X_train, y_train)

In [67]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [68]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [69]:
clf.fit(X_train, y_train)
my_clf.fit(X_test, y_test)
accuracy_score(y_pred=clf.predict(X_test), y_true=y_test)

0.8888888888888888

In [70]:
%time clf.fit(X_train, y_train)

Wall time: 4 ms


DecisionTreeClassifier()

In [71]:
%time my_clf.fit(X_train, y_train)

Wall time: 47 ms


In [72]:
accuracy_score(y_pred=clf.predict(X_test), y_true=y_test)

0.8888888888888888

In [73]:
accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

0.8888888888888888

## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [74]:
%time clf.fit(X_train, y_train)

Wall time: 5 ms


DecisionTreeClassifier()

In [75]:
%time my_clf.fit(X_train, y_train)

Wall time: 44 ms


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [76]:
df = pd.read_csv('Speed Dating Data.csv', encoding='latin1')

In [77]:
df = df.iloc[:, :97]
df = df.drop(['id', 'idg', 'condtn', 'round', 'position', 'positin1',
             'order', 'partner', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin',
              'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o',
              'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o',
              'like_o', 'prob_o', 'met_o', 'field', 'undergra',
              'imprelig', 'imprace', 'from', 'zipcode', 'career', 'sports',
              'tvsports', 'exercise', 'dining', 'museums',
              'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
              'theater', 'movies', 'concerts', 'music', 'shopping',
              'yoga', 'expnum', 'wave'], axis=1)
df = df.dropna(subset=['age'])
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
df = pd.get_dummies(df, columns=['field_cd'],
                    prefix='field_cd', prefix_sep='=')
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.mn_sat = df.mn_sat.fillna(-999)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
df.tuition = df.tuition.fillna(-999)
#df = pd.get_dummies(df, columns=['race'],
#                    prefix='race', prefix_sep='=')
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
df = df.dropna(subset=['date'])

df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
df = pd.get_dummies(df, columns=['career_c'],
                    prefix='career_c', prefix_sep='=')

a = ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
df.loc[:, 'temp_totalsum'] = df.loc[:, a].sum(axis=1)
df.loc[:, a] =\
    (df.loc[:, a].T / df.loc[:, 'temp_totalsum'].T).T * 100
a = ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']
df.loc[:, 'temp_totalsum'] = df.loc[:, a].sum(axis=1)
df.loc[:, a] =\
    (df.loc[:, a].T / df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)

for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
if i != 4:
    feat.remove('shar{}_1'.format(i))
    
df = df.drop(feat, axis=1)

df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)
df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
df_pair = df_pair.drop(['iid', 'pid'], axis=1)
X = df_pair.iloc[:, 1:].values
y = df_pair.iloc[:, 0].values

X, y

(array([[ 1.6e-01,  0.0e+00,  2.6e+01, ...,  0.0e+00,  0.0e+00,  0.0e+00],
        [ 3.4e-01,  1.0e+00,  3.2e+01, ...,  0.0e+00,  0.0e+00,  0.0e+00],
        [ 5.2e-01,  0.0e+00,  3.7e+01, ...,  0.0e+00,  0.0e+00,  0.0e+00],
        ...,
        [-2.0e-01,  0.0e+00,  3.0e+01, ...,  0.0e+00,  0.0e+00,  0.0e+00],
        [-3.2e-01,  0.0e+00,  2.7e+01, ...,  0.0e+00,  0.0e+00,  0.0e+00],
        [ 1.0e-02,  0.0e+00,  2.5e+01, ...,  0.0e+00,  0.0e+00,  0.0e+00]]),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

In [78]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [80]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [81]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5545367127412596

In [82]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.48458649323221653

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

In [46]:
clf.feature_importances_

array([0.06481359, 0.        , 0.02743493, 0.00388221, 0.00468113,
       0.00460891, 0.00641711, 0.0060539 , 0.02215313, 0.00668841,
       0.01912074, 0.01343629, 0.01904423, 0.0123299 , 0.01015479,
       0.00741294, 0.00947645, 0.02953333, 0.01795361, 0.0100657 ,
       0.02933137, 0.02257429, 0.01113297, 0.01087976, 0.03277882,
       0.02159898, 0.02002085, 0.        , 0.03510456, 0.00761093,
       0.01586958, 0.00387475, 0.        , 0.0089573 , 0.        ,
       0.00793816, 0.        , 0.        , 0.        , 0.0102211 ,
       0.        , 0.00158894, 0.        , 0.00040959, 0.        ,
       0.        , 0.        , 0.00371261, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00953107, 0.00284591, 0.        ,
       0.00226506, 0.        , 0.00277776, 0.00226506, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.02070943, 0.01034024, 0.01777324, 0.00148

In [47]:
my_clf.feature_importances_

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

